# Searching for Dwarf Galaxies in Pan-STARRS

## Learning Goals

By the end of this tutorial you will: <br>
* Learn to search a survey for observations of a known target.
* Be able to plot a color-magnitude diagram.
* Understand how to find and plot an image of a field of view. 
* Use coordinates to download the data products of a known member in a dwarf galaxy. 

## Introduction

There are a number of dwarf galaxies that are gravitationally bound to the Milky Way. These satellites are small and generally faint companions in the Local Group that are not observable with the naked eye, except for the Large and Small Magenllanic Cloud. These satellites are classified into two major classes: Classical dwarf spheroidal and ultra-faint dwarf (UDF) galaxies. Classical dwarf spheroidal galaxies are small, low-luminosity galaxies with very little dust and an older stellar population ([Metz et al. 2018](https://ui.adsabs.harvard.edu/abs/2007MNRAS.376..387M/abstract)). Ultra-faint dwarf galaxies contain from a few hundred to one hundred thousand stars, making them the faintest galaxies in the Universe ([Simon et al. 2019](https://www.annualreviews.org/doi/10.1146/annurev-astro-091918-104453)).

We will be searching the [Pan-STARRS](https://outerspace.stsci.edu/display/PANSTARRS/) catalog for data on an already discovered dwarf spheroidal galaxy and then try to search if there are other MAST data associated with the galaxy. The methods we use in this notebook can be extended to find new dwarf galaxies but in this application we will be using the MAST archive and its tools to search PS1 for data on an already known target. To choose our target, we will be referencing the paper "Milky Way Satellite Census. I. The Observational Selection Function for Milky Way Satellites in DES Y3 and Pan-STARRS DR1" by [Drlica-Wagner et al. 2020](https://iopscience.iop.org/article/10.3847/1538-4357/ab7eb9).

After searching PS1 for data on our target, we will use the data to plot a color-magnitude diagram of the galaxy and also plot an image of the field of view of our target. Next, we will choose another target (a UFD galaxy) repeat the steps of gathering PS1 data, and compare both target's color-magnitude diagrams. Lastly, we will search Hubble Space Telescope data to download the data products available for our first target. 


## Imports

The following cell holds the imported packages. These packages are necessary for running the rest of the cells in this notebook. A description of each import is as follows:

* numpy to handle array functions
* fits from astropy.io for accessing FITS files
* Table from astropy.table for creating tidy tables of the data
* Image from PIL to functions to load images from a URL
* ascii from astropy.io for accessing the API results
* matplotlib.pyplot for plotting data
* Mast, Observations, and Catalogs from astroquery.mast for querying data and observations from the MAST archive
* requests for gathering data from the PS1 API
* BytesIO is needed to manipulate bytes data in memory that are returned by requests.
* units from astropy and SkyCoord from astropy.coordinates for converting RA and Dec into degrees

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import requests
from io import BytesIO
from PIL import Image
from astropy.io import fits
import astropy.utils
from astropy.table import Table
from astroquery.mast import Mast, Observations, Catalogs
from astropy.io import ascii
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS

# Searching PS1 Data for a Known Target

## Choosing a Target

Using Table 2 and Table 4 from Drlica-Wagner et al. 2020, we can choose a galaxy that has PS1 data to search for. We will choose the galaxy with the brightest magnitude that has been confirmed using PS1: **Leo I**. 

Table 2 lists the Right Acension, Declination, and distance of **Leo I** which we will use to search the database. We can see that it's basic characteristics are: 
* Right Ascension: 10h 08m 28.12s or 152.1146 degrees
* Declination: +12deg 18' 23.4" or 12.3059 degrees
* Distance: 254 kiloparsecs 
* Magnitude in v band: −11.78 

### Gather necessary parameters 

Table 2 lists the Right Acension, Declination, and distance of **Leo I** which we will use to search the database. Additionally, we will need the API url in order to search PS1 for this target. We are using a search radius around **Leo I** of 15 arcminutes to be sure we are looking at the entire galaxy, [SIMBAD](https://simbad.u-strasbg.fr/simbad/sim-id?Ident=leo+I&NbIdent=1&Radius=2&Radius.unit=arcmin&submit=submit+id) lists the angular size of **Leo I** as 11.5 arcminutes. 

In [ ]:
ra = "152.1146" #[deg]
dec = "12.3059" #[deg]
dist = "254" #[kpc]
radius = "0.25" #[deg] = 20 arcminutes

In [ ]:
baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs/" #this is the base url for querying the API
release = "dr1/" #you can also look at dr2 if you want
obj = "mean" #we will look at the mean photometric information for our target
condition = "nDetections.gte=1" #this condition allows us to only get data where at least 1 observation has been made
form = "csv"

## Retrieve PS1 data

The Panoramic Survey Telescope & Rapid Response System (Pan-STARRS) is a wide-field imaging facility developed at the University of Hawaii's Institute for Astronomy. We are going to be using the MAST API for PS1 to gather our data. Discriptions for the PS1 MeanObjectView table fields can be found here: https://outerspace.stsci.edu/display/PANSTARRS/PS1+MeanObjectView+table+fields . 

### Build API URL and Get Data

In [ ]:
#now build the url to get the data using the parameters we gathered
url = baseurl+release+obj+"?ra="+ra+"&dec="+dec+"&radius="+radius+"&"+condition+"&format="+form
print(url)

#Request data from the API
results = requests.get(url)
results = results.text

#Make a table from the results
tab = ascii.read(results)
tab[0:5]

## Plot Color-Magnitude Diagram (CMD) of Leo I

Color-Magnitude Diagrams (CMDs) are a common way astronomers visualize the photometric parameters of a stellar system. Where the stars of a system fall on a CMD can tell you things like the age of that system and therefore, what types of stars it is composed of. The colors of a system can also tell us about the temperature of that system. Color in astronomy is defined to be the difference between the magnitude in one filter and the magnitude in another filter.

### Gather arrays for plotting
A CMD consists of the magnitude in one filter (vertical-axis) vs the difference between the magnitude in that filter and the magnitude in another filter (horizontal-axis). For this example we will plot the magnitude of **Leo I** in g vs the magnitude of **Leo I** in g-r. Additionally, we'll take a look at the CMD in r-i so that you can see how to plot multiple types of CMDs for future reference.

What CMDs show is the relationship between the magnitude of the signal we have detected in different color filters, so in this case a larger value for g (bluer filter) minus r (red filter) means the star is redder, and a smaller g-r value means the star is bluer. A larger value in r (red) minus i (even more red) means the star is redder, and a smaller r-i value means the star is bluer. In order to plot these CMDs we will need a few numpy arrarys to store these magnitudes (g,r,i).


In [ ]:
g_mag = tab["gMeanApMag"] #mean aperture magntiude in g filter
r_mag = tab["rMeanApMag"] #mean aperture magntiude in r filter
i_mag = tab["iMeanApMag"] #mean aperture magntiude in i filter

#convert to numpy arrays for easy plotting
g_mag = np.array(g_mag)
r_mag = np.array(r_mag)
i_mag = np.array(i_mag)

#get the difference between the filters 
g_r = np.subtract(g_mag,r_mag)

#get the difference between the filters 
r_i = np.subtract(r_mag,i_mag)

### Plot Total CMD in g - r

In [ ]:
plt.scatter(g_r, g_mag, c = "coral", edgecolor = "black", alpha = 0.75)
plt.xlim(-5,5)
plt.ylim(28,12.5)
plt.xlabel("g - r")
plt.ylabel("g")

### Filter Data to Show CMD with Old Stars 
The CMD of all of **Leo I** doesn't tell us much about the stellar evolution of the system. Let's filter the data so we can compare the whole galaxy with just its outer stars that are old and metal-poor.

First, we need to calculate the distance modulus of **Leo I**. A distance modulus is the difference between the apparent magnitude and absolute magnitude of a celestial object (m – M), and provides a measure of the distance to the object, r. For **Leo I** (m-M) = 21.8. [Drlica-Wagner et al. 2020](https://iopscience.iop.org/article/10.3847/1538-4357/ab7eb9) shows in Section 5.1, that metal-poor stars reside in a distance modulus between 16 and 22. We know from [Held et al. 2000](https://iopscience.iop.org/article/10.1086/312505/pdf) that old metal-poor stars in **Leo I** will be outside a radius of 4.5' from the galactic center.

In [ ]:
#Filter data to be outside radius of 4.5' = 0.075 degrees
ind = np.where((tab["raMean"] <= float(ra)+0.075) & (tab["decMean"] <= float(dec)+0.075))
g_mag_outer = tab["gMeanApMag"][ind] #mean aperture magntiude in g filter for just outer stars
r_mag_outer = tab["rMeanApMag"][ind] #mean aperture magntiude in r filter for just outer stars
i_mag_outer = tab["iMeanApMag"][ind] #mean aperture magntiude in i filter for just outer stars
g_r_outer = g_mag_outer-r_mag_outer
r_i_outer = r_mag_outer-i_mag_outer

#Filter the data again to be of a distance modulus between 16 and 22
# distance_modulus = 11.2-tab["gMeanApMag"][ind]
# ind2 = np.where((distance_modulus <= 22) & (distance_modulus>=16)) 
# g_mag_outer = g_mag_outer[ind2] #mean aperture magntiude in g filter for just outer stars
# r_mag_outer = r_mag_outer[ind2] #mean aperture magntiude in r filter for just outer stars
# g_r_outer = g_mag_outer-r_mag_outer

### Overplot CMD of Whole Galaxy with CMD of just Outer Stars

In [ ]:
plt.scatter(g_r, g_mag, c = "coral", edgecolor = "black", alpha = 0.75)
plt.scatter(g_r_outer, g_mag_outer, c = "darkgrey", s = 5)
plt.xlim(-5,5)
plt.ylim(28,12.5)
plt.xlabel("g - r")
plt.ylabel("g")

### Plot CMD in r - i

In [ ]:
plt.scatter(r_i, i_mag, c = "turquoise", edgecolor = "black", alpha = 0.75)
plt.xlim(-5,5)
plt.ylim(30,10)
plt.xlabel("r - i")
plt.ylabel("i")

### Overplot CMD of Whole Galaxy with CMD of Outer Stars in r-i

In [ ]:
plt.scatter(r_i, i_mag, c = "turquoise", edgecolor = "black", alpha = 0.75)
plt.scatter(r_i_outer, i_mag_outer, c = "darkgrey", s = 5)
plt.xlim(-5,5)
plt.ylim(30,10)
plt.xlabel("r - i")
plt.ylabel("i")

### Plot CMD in g-r and r-i side-by-side

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2)
fig.set_size_inches(10,3)

ax1.scatter(g_r, g_mag, c = "coral", edgecolor = "black", alpha = 0.75)
ax1.set_xlim(-5,5)
ax1.set_ylim(30,10)
ax1.set_xlabel("g - r")
ax1.set_ylabel("g")

ax2.scatter(r_i, i_mag, c = "turquoise", edgecolor = "black", alpha = 0.75)
ax2.set_xlim(-5,5)
ax2.set_ylim(30,10)
ax2.set_xlabel("r - i")
ax2.set_ylabel("i")

## Plot Field of View (FOV) of Leo I

### Build the url to the fits file of the region for our target by the cutout service.


In [ ]:
filters = "grizy"
size = "3000" #extracted image size in pixels (0.25 arcsec/pixel)
form = "fits" #file format of the file we would like

color = "red" #you can change this, red is typically the i or r filter

#get the url to the PS1 images
images_url = "https://ps1images.stsci.edu/cgi-bin/ps1filenames.py?ra="+ra+"&dec="+dec+"&filters="+filters
#read the images data into a table
img_table = Table.read(images_url, format='ascii')

#get the first filename from the list of images
file = img_table['filename'][0]

#build the url to the fits file
fits_url = "https://ps1images.stsci.edu/cgi-bin/fitscut.cgi?ra="+ra+"&dec="+dec+"&size="+size+"&format="+form+"&"+color+"="+file

print(fits_url)

### Open the fits file and inspect the header

Here we will plot the field of view containing our galaxy, **Leo I**. In the next step we will also show you how to convert from pixel coordinates to sky coordinates (RA and Dec), to do this we will use the information in the FITS header combined with the astropy package WCS. FITS headers contain meta-deta about the image or data cube including but not limited to: the date, time, telescope, and instrument that made the observation, the exposure time and wavelength band of the data, and the physical size and binning of the pixels in the image.

In [ ]:
#open the fits file and get the data
fh = fits.open(fits_url)
fits_img = fh[0].data

#get the FITS header
header = fh[0].header
wcs_leoI = WCS(header)

#take a look at the FITS header
header

### Plot the Field of View image

In [ ]:
# replace NaN values with zero for display
fits_img[np.isnan(fits_img)] = 0.0

#make figure
fig = plt.figure(figsize = (10, 10))
#this subplot with the "projection" keyword will put our axes into units of RA and Dec
ax = plt.subplot(projection = wcs_leoI)
plt.imshow(fits_img, origin = 'lower', cmap = 'Greys_r', aspect = 'equal', vmin = 0, vmax = 255)
plt.xlabel('RA')
plt.ylabel('Dec')
plt.xlim(400,3000)

### Build the url to the color image file of the region for our target

In [ ]:
size = "3000" #extracted image size in pixels (0.25 arcsec/pixel) = 12.5 arcminutes of angular size
form = "png" #file format of the file we would like, for color images it has to be a jpeg or png
filters = "yig" #using three images of filters 'y' (for red), 'i' (for green), and 'g' (for blue). 

#build the PS1 url for the color image
color_img_url = "https://ps1images.stsci.edu/cgi-bin/ps1filenames.py?ra="+ra+"&dec="+dec+"&filters="+filters
color_img_table = Table.read(color_img_url, format='ascii')
color_file = color_img_table['filename'][0]
color_png_url = "https://ps1images.stsci.edu/cgi-bin/fitscut.cgi?ra="+ra+"&dec="+dec+"&size="+size+"&format="+form

#now we need to get all three colors (r,g,b)
for i, param in enumerate(["red","green","blue"]):
            color_png_url = color_png_url + "&{}={}".format(param,color_img_table['filename'][i])


### Plot the FOV in Color 

In [ ]:
#open the url and get the image
r = requests.get(color_png_url)
im = Image.open(BytesIO(r.content))
#make figure
fig = plt.figure(figsize = (10, 10))
#this subplot with the "projection" keyword will put our axes into units of RA and Dec
ax = plt.subplot(projection = wcs_leoI)
#plot the image
plt.imshow(im, origin = 'lower', aspect = 'equal',vmin=0, vmax = 255) 
plt.xlabel(r'RA')
plt.ylabel(r'Dec')
plt.xlim(400,3000)

## Repeat with New Target and Compare

Now we will repeat the same process for a new target, **Draco II**, which is a UFD galaxy and compare it with **Leo I**, a classical dwarf spheroidal.

### Retrieve PS1 data 
This is the same method as before, just using the position of **Draco II** instead. You can find the RA and Dec of **Draco II** in Table I of its discovery paper, [Laevens et al 2015](https://iopscience.iop.org/article/10.1088/0004-637X/813/1/44 ). It is also listed on [SIMBAD](https://simbad.u-strasbg.fr/simbad/sim-id?Ident=draco+ii&NbIdent=1&Radius=2&Radius.unit=arcmin&submit=submit+id). 

In [ ]:
#read RA and Dec from either SIMBAD or Table 1 in Laevens et al 2015
r = '15:52:47.6'
d = '+64:33:55'
form = "csv"
radius = "0.1"

In [ ]:

#Convert the RA and Dec to degrees
c = SkyCoord(r+d, unit=(u.hourangle, u.deg))
draco_ra = str(c.ra.degree)
draco_dec = str(c.dec.degree)

#now build the url to get the data from using the parameters we gathered
draco_url = baseurl+release+obj+"?ra="+draco_ra+"&dec="+draco_dec+"&radius="+radius+"&"+condition+"&format="+form
print(draco_url)

#Request data from the API
draco_results = requests.get(draco_url)
draco_results = draco_results.text

#Make a table from the results
draco_tab = ascii.read(draco_results)
draco_tab[0:5]

## Plot CMDs of Draco II

Gather the arrays for plotting the color-magnitude diagrams using the same method as before. 

In [ ]:
draco_g_mag = draco_tab["gMeanApMag"] #mean aperture magntiude in g filter
draco_r_mag = draco_tab["rMeanApMag"] #mean aperture magntiude in r filter
draco_i_mag = draco_tab["iMeanApMag"] #mean aperture magntiude in i filter

#convert to numpy arrays for easy plotting
draco_g_mag = np.array(draco_g_mag)
draco_r_mag = np.array(draco_r_mag)
draco_i_mag = np.array(draco_i_mag)

#get the difference between the filters 
draco_g_r = np.subtract(draco_g_mag,draco_r_mag)
draco_r_i = np.subtract(draco_r_mag,draco_i_mag)

### Plot CMD in g-r and r-i side-by-side

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2)
fig.set_size_inches(10,3)

ax1.scatter(draco_g_r, draco_g_mag, c = "gold", edgecolor = "black", alpha = 0.75)
ax1.set_xlim(-5,5)
ax1.set_ylim(30,10)
ax1.set_xlabel("g - r")
ax1.set_ylabel("g")

ax2.scatter(draco_r_i, draco_i_mag, c = "dodgerblue", edgecolor = "black", alpha = 0.75)
ax2.set_xlim(-5,5)
ax2.set_ylim(30,10)
ax2.set_xlabel("r - i")
ax2.set_ylabel("i")

## Let's overplot the CMDs for Leo I and Draco II 

To compare the compositions of **Leo I** and **Draco II**, we can plot their Color-Magnitude Diagrams on top of each other to see how they differ. As before, we will first plot a CMD in g-r vs g and then in r-i vs i. 

In [ ]:
plt.scatter(g_r, g_mag, c = "coral", label = "Leo I", edgecolor = "black", alpha = 0.75)
plt.scatter(draco_g_r, draco_g_mag, c = "gold", label = "Draco II", edgecolor = "black", alpha = 0.75)
plt.xlim(-5,5)
plt.ylim(30,10)
plt.xlabel("g - r")
plt.ylabel("g")
plt.legend()

In [ ]:
plt.scatter(r_i, i_mag, c = "turquoise", label = "Leo I", edgecolor = "black", alpha = 0.75)
plt.scatter(draco_r_i, draco_i_mag, c = "dodgerblue", label = "Draco II", edgecolor = "black", alpha = 0.75)
plt.xlim(-5,5)
plt.ylim(30,10)
plt.xlabel("r - i")
plt.ylabel("i")
plt.legend()

We can see that the CMDs for both galaxies are actually quite similar. They show the same distribtion in either color. The average color of **Draco II's** stars look to be very slightly redder than **Leo I**, at least in the g-r CMD. **Draco II** is also a smaller galaxy with less stars -- this is expected since **Draco II** is an ultra-faint dwarf while **Leo I** is a classical dwarf. 

## Find the Spectral Image for a Member of Leo I
 
We can use the paper [Kirby et al. 2010 ](https://ui.adsabs.harvard.edu/abs/2010ApJS..191..352K/abstract) to find an individual member of **Leo I**. From here we can use the coordinates of a member, search the MAST archive for observations of it, and download its spectrum.

Let's take a look at member **LeoI72134** by Kirby et al. 2010, we are choosing this star because it has multiple citations so we should be able to learn a lot about it. 

### Convert RA and Dec to Degrees

The coordinates from Kirby et al. 2010 are in units of hour angle while the MAST archive needs units of degrees, so we will have to convert the coordinates.

In [ ]:
#Get the RA and Dec
ra = "10 08 01.88"
dec = "+12 17 55.5" 

#Convert the RA and Dec into degrees
c = SkyCoord(ra+dec, unit=(u.hourangle, u.deg))
ra = str(c.ra.deg)
dec = str(c.dec.deg)
print(ra,dec)

### First, Query MAST Archive for HST observations of LeoI72134

In [ ]:
hst_table = Observations.query_criteria(coordinates = ra+dec,radius="0.5 degrees", dataproduct_type="spectrum", obs_collection="HST")

#Let's print out some relevant columns of this table
hst_table["instrument_name","filters","target_name","obs_id","calib_level","t_exptime"]

### Now, we have to select observations for a specific instrument and filter combination

Unfortuntely, there are no MAST data of any specific stars in **Leo I**. Here we will show the process for the object that we do find an observation for, but keep in mind this object is not inside **Leo I** but is far away from the galaxy in the same region of the sky within a 0.5 degree radius. 

In [ ]:
g230L_table = hst_table['obsid','obs_id','target_name','calib_level',
                        't_exptime','filters','em_min','em_max'][hst_table['filters']=='G230L']

sel_table = g230L_table[np.argmin(g230L_table['t_exptime'])]
sel_table

### Get the Minimum Recommended Products for this observation

Now, we will query the observations from MAST to get a list of products for our selected observation. Then, we will gather the minimum recommended products needed in order to plot a spectrum. This step is necessary as HST specifically can provide more data than is needed to plot the spectrum of our observation. 

In [ ]:
data_products = Observations.get_product_list(sel_table)

wprod = np.where((data_products['productGroupDescription']=='Minimum Recommended Products') & 
                 (data_products['productType']=='SCIENCE'))[0]

data_products[wprod]

### Download the data to plot

In [ ]:
data = Observations.download_products(data_products[wprod])

data

### Next, we will investigate the FITS file we just downloaded 

In [ ]:
filename = data['Local Path'][0]
hdulist = fits.open(filename)

header = hdulist[0].header
scidata = hdulist[1].data

### Vizualize the Spectral Image

Now, finally, we can investiage the spectral image using Matplotlib.

In [ ]:
plt.rcParams["figure.figsize"] = (20,3)
plt.imshow(scidata, cmap="CMRmap")
plt.ylim(600,450)
plt.xlabel("x pixels")
plt.ylabel("y pixels")

## Exercise

### 1. Plot Field of View of Draco II 

Can you recreate the steps we took for **Leo I** to plot the field of view of **Draco II** in black and white and in color? 

# About this Notebook

**Author:** Emma Lieb <br>
**Last Updated:** Sept 2022

# Citations

* [Citation for `astropy`](https://www.astropy.org/acknowledging.html)